In [1]:
import requests
from bs4 import BeautifulSoup
import csv # CSV 저장을 위해 csv 모듈 추가
import re
from datetime import datetime # 날짜 처리를 위해 datetime 모듈 추가

# 숭실대학교 스캐치(Scatch) 공지사항 페이지 URL을 설정합니다.
# 실제 공지사항 목록 페이지 URL로 대체해야 합니다.
# 예시 URL은 목록 페이지의 첫 번째 페이지를 가정합니다.
BASE_URL = "https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/" # <- 이 부분을 실제 URL로 변경해주세요.

def fetch_notices(url):
    """
    지정된 URL에서 공지사항 목록을 가져와 파싱하고 2025년 이후의 항목만 필터링합니다.
    """
    # HTML 요청 시 발생할 수 있는 오류를 처리하기 위해 try-except 블록을 사용합니다.
    try:
        # 사용자 에이전트를 설정하여 봇 감지를 회피하고 정상적인 접근을 시도합니다.
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status() # HTTP 오류가 발생하면 예외를 발생시킵니다.

        # 인코딩 문제 방지를 위해 응답 텍스트를 파싱합니다.
        soup = BeautifulSoup(response.text, 'html.parser')

    except requests.exceptions.RequestException as e:
        print(f"웹 페이지를 가져오는 중 오류가 발생했습니다: {e}")
        return []

    # 공지사항 리스트를 담을 빈 리스트를 초기화합니다.
    notices_data = []

    # 각 공지사항 항목을 찾습니다. (<li> 태그의 하위 요소인 div.row...를 찾음)
    notice_items = soup.find_all('div', class_=lambda c: c and 'row no-gutters align-items-center' in c)

    for item in notice_items:
        try:
            # 1. 작성일 추출 및 필터링 (2025년 이후)
            date_element = item.select_one('.notice_col1 .h2')
            date = date_element.text.strip() if date_element else None

            if date:
                # 'YYYY.MM.DD' 형식에서 연도만 추출하여 필터링
                try:
                    # date[0:4]는 'YYYY' 부분입니다.
                    year = int(date[0:4])
                    if year < 2025:
                        # 2025년 이전의 공지사항은 건너뜁니다.
                        continue
                except ValueError:
                    # 날짜 형식이 잘못된 경우 건너뛰거나 'N/A'로 처리할 수 있습니다.
                    print(f"경고: 작성일 형식 오류로 필터링을 건너pro니다. (Date: {date})")
                    continue
            else:
                date = 'N/A'
                continue # 날짜가 없는 항목은 처리하지 않음

            # 2. 진행상황 추출
            status_element = item.select_one('.notice_col2 .tag')
            status = status_element.text.strip() if status_element else 'N/A'

            # 3. 카테고리, 제목 및 링크 추출 (notice_col3 내부)
            link_element = item.select_one('.notice_col3 a')
            if link_element:
                link_href = link_element['href'].strip()

                category_element = link_element.select_one('.label')
                category = category_element.text.strip() if category_element else 'N/A'

                # 제목 추출
                title_span = link_element.select_one('span[class*="d-inline-b"].m-pt-5')
                title = title_span.text.strip() if title_span else '제목 없음 (파싱 오류)'

            else:
                link_href, category, title = 'N/A', 'N/A', 'N/A'

            # 4. 등록부서 추출 (notice_col4)
            department_element = item.select_one('.notice_col4')
            department = department_element.text.strip() if department_element else 'N/A'

            # 5. 조회수 추출 (notice_col5)
            views_element = item.select_one('.notice_col5')
            # 콤마(,) 등 불필요한 문자를 제거하고 숫자만 추출하거나 원본 그대로 저장
            views = views_element.text.strip() if views_element else 'N/A'

            # 추출된 데이터를 딕셔너리 형태로 저장합니다.
            notices_data.append({
                '작성일': date,
                '진행상황': status,
                '카테고리': category,
                '제목': title,
                '링크': link_href,
                '등록부서': department, # <-- 신규 항목
                '조회수': views       # <-- 신규 항목
            })

        except Exception as e:
            print(f"개별 공지사항 항목 파싱 중 오류 발생: {e}")
            continue

    return notices_data

def save_to_csv(data, filename="ssu_notices_2025_onwards.csv"):
    """
    크롤링한 데이터를 CSV 파일로 저장합니다.
    """
    if not data:
        print("저장할 데이터가 없습니다.")
        return

    # CSV 파일에 사용할 헤더(필드 이름)를 설정합니다. (신규 항목 포함)
    fieldnames = ['작성일', '진행상황', '카테고리', '제목', '링크', '등록부서', '조회수']

    # 파일을 쓰기 모드로 열고 인코딩은 'utf-8-sig'를 사용하여
    # 엑셀에서 한글 깨짐을 방지합니다.
    try:
        with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            # 헤더를 먼저 작성합니다.
            writer.writeheader()

            # 데이터 행들을 작성합니다.
            writer.writerows(data)

        print(f"\n데이터가 '{filename}' 파일에 성공적으로 저장되었습니다.")
    except Exception as e:
        print(f"\nCSV 저장 중 오류가 발생했습니다: {e}")

if __name__ == '__main__':
    print("--- 숭실대학교 공지사항 크롤링 시작 (2025년 이후 필터링) ---")

    # 크롤링 함수를 실행하여 데이터를 가져옵니다.
    results = fetch_notices(BASE_URL)

    if results:
        print(f"총 {len(results)}개의 (2025년 이후) 공지사항을 찾았습니다.")

        # 크롤링 결과 출력 (처음 5개 항목만 출력)
        for i, notice in enumerate(results[:5]):
            print(f"--- 항목 {i+1} ---")
            print(f"작성일: {notice['작성일']}")
            print(f"진행상황: {notice['진행상황']}")
            print(f"카테고리: {notice['카테고리']}")
            print(f"제목: {notice['제목']}")
            print(f"등록부서: {notice['등록부서']}") # <-- 출력 추가
            print(f"조회수: {notice['조회수']}")   # <-- 출력 추가
            print(f"링크: {notice['링크']}\n")

        if len(results) > 5:
            print(f"...")

        # CSV 파일로 저장
        save_to_csv(results)

    else:
        print("공지사항을 찾지 못했거나 크롤링에 실패했습니다. URL과 CSS Selector를 다시 확인해 주세요.")

    print("--- 크롤링 및 CSV 저장 완료 ---")

--- 숭실대학교 공지사항 크롤링 시작 (2025년 이후 필터링) ---
총 15개의 (2025년 이후) 공지사항을 찾았습니다.
--- 항목 1 ---
작성일: 2025.11.04
진행상황: 진행
카테고리: 비교과·행사
제목: [중앙도서관] ‘천선란’ 작가 저자강연회 참가자 모집(~11/21)
등록부서: 중앙도서관
조회수: 40
링크: https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/?f&category&paged=1&slug=%EC%A4%91%EC%95%99%EB%8F%84%EC%84%9C%EA%B4%80-%EC%B2%9C%EC%84%A0%EB%9E%80-%EC%9E%91%EA%B0%80-%EC%A0%80%EC%9E%90%EA%B0%95%EC%97%B0%ED%9A%8C-%EC%B0%B8%EA%B0%80%EC%9E%90-%EB%AA%A8&keyword

--- 항목 2 ---
작성일: 2025.11.04
진행상황: 진행
카테고리: 국제교류
제목: 2025학년도 2학기 SIC 4쿼터 정기 교환학생 세미나 (SAF)
등록부서: 국제팀
조회수: 47
링크: https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/?f&category&paged=1&slug=2025%ED%95%99%EB%85%84%EB%8F%84-2%ED%95%99%EA%B8%B0-sic-4%EC%BF%BC%ED%84%B0-%EC%A0%95%EA%B8%B0-%EA%B5%90%ED%99%98%ED%95%99%EC%83%9D-%EC%84%B8%EB%AF%B8%EB%82%98-saf&keyword

--- 항목 3 ---
작성일: 2025.11.03
진행상황: 진행
카테고리: 장학
제목: 2025년 우양재단 동행 장학생 모집 안내
등록부서: 장학팀
조회수: 196
링크: https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/?f&ca